# Latent Dirichlet Allocation

In [1]:
import pandas as pd

df = pd.read_csv('data', sep=",", header=None)

df.columns = ['text']

df.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


The data is a collection of emails that are not labelled. Let's try extract topics from them!

## Preprocessing 

👇 You're used to it by now... Clean up! Store the cleaned text in a new dataframe column "clean_text".

In [2]:
import string
import re
#fonction permettant d'enlever la ponctuatuion des mots
def remove_punctuation(text):
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub("\d+", " ", text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.lower()
    
    return text

In [3]:
#Fonction qui permet de retirer les chiffres grace a isdigit qui parcourt les mots et supprime les chiffre
def remove_numbers(text):
    return ''.join(c for c in text if not c.isdigit())

In [4]:
df["clean_text"] = df["text"].apply(remove_punctuation)
df

,text,clean_text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,from gldcunixbcccolumbiaedu gary l dare subjec...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...,from atterlepvelaacsoaklandedu cardinal ximene...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...,from minerkuhubccukansedu subject re ancient b...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...,from atterlepvelaacsoaklandedu cardinal ximene...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...,from vzhivovsuperiorcarletonca vladimir zhivov...
...,...,...
1194,From: jerryb@eskimo.com (Jerry Kaufman)\nSubje...,from jerrybeskimocom jerry kaufman subject re ...
1195,From: golchowy@alchemy.chem.utoronto.ca (Geral...,from golchowyalchemychemutorontoca gerald olch...
1196,From: jayne@mmalt.guild.org (Jayne Kulikauskas...,from jaynemmaltguildorg jayne kulikauskas subj...
1197,From: sclark@epas.utoronto.ca (Susan Clark)\nS...,from sclarkepasutorontoca susan clark subject ...


## Latent Dirichlet Allocation model

👇 Train an LDA model to extract potential topics.

In [5]:
import gensim
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel

dictionary = corpora.Dictionary(df['clean_text'].str.split().tolist())

doc_term_matrix = [dictionary.doc2bow(text) for text in df['clean_text'].str.split().tolist()]

# creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# runnug and trainig LDA model on the document term matrix for 3 topics
ldamodel = Lda(doc_term_matrix, num_topics = 3, id2word = dictionary, passes = 50)

dictionary

## Visualize potential topics

👇 The function to print the words associated with the potential topics is already made for you. You just have to pass the correct arguments!

In [6]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, doc_term_matrix, dictionary)
vis

C:\Users\utilisateur\anaconda3\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
C:\Users\utilisateur\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.120687 -0.036317       1        1  57.369856
1      0.129698 -0.029238       2        1  22.702731
2     -0.009012  0.065555       3        1  19.927412, topic_info=     Term          Freq         Total Category  logprob  loglift
152   the  18233.000000  18233.000000  Default  30.0000  30.0000
240     i   4806.000000   4806.000000  Default  29.0000  29.0000
0       a   6373.000000   6373.000000  Default  28.0000  28.0000
63     in   6208.000000   6208.000000  Default  27.0000  27.0000
327   you   2648.000000   2648.000000  Default  26.0000  26.0000
..    ...           ...           ...      ...      ...      ...
19    but    353.666408   1955.156298   Topic3  -5.3393  -0.0968
309  this    379.605797   2394.841104   Topic3  -5.2685  -0.2289
573    he    342.130215   2036.595819   Topic3  -5.3724  -0.1708
13     be    353.320957   2470.932713   Topic3  -5.3403  -0.3319
97    not    350.954580   2620.239992   Topic3  -5.3470  -0.3973

[280 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         1  0.592905           a
0         2  0.201296           a
0         3  0.205689           a
2810      2  0.980212  adirondack
2661      2  0.995586         ahl
...     ...       ...         ...
881       3  0.270302        year
327       1  0.703716         you
327       2  0.035488         you
327       3  0.260873         you
645       2  0.995448        zone

[399 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

## Predict topic of new text

👇 You can now use your LDA model to predict the topic of a new text. First, use your vectorizer to vectorize the example. Then, use your LDA model to predict the topic of the vectorized example.

In [7]:
new_text = ["god, the book of truth, an epic saga by the famous author patrick rothfuss"]
vectorizer = TfidfVectorizer().fit(new_text)
df_vectorized = vectorizer.transform(new_text)
ldamodel = LatentDirichletAllocation(n_components=2).fit(df_vectorized)
print_topics(ldamodel, vectorizer)

NameError: name 'TfidfVectorizer' is not defined